# 题目

> 中位数是有序整数列表中的中间值。如果列表的大小是偶数，则没有中间值，中位数是两个中间值的平均值。
例如 arr = [2,3,4] 的中位数是 3 。  
例如 arr = [2,3] 的中位数是 (2 + 3) / 2 = 2.5 。  

> 实现 MedianFinder 类:  
MedianFinder() 初始化 MedianFinder 对象。  
void addNum(int num) 将数据流中的整数 num 添加到数据结构中。  
double findMedian() 返回到目前为止所有元素的中位数。与实际答案相差 10-5 以内的答案将被接受。

# 方法一：二分查找

> 使用两个堆维护所有新加入的数：  
小顶堆queMax中所有数都大于中位数；  
大顶堆queMin中所有数都小于等于中位数。  
大顶堆queMin的长度始终大于等于小顶堆queMax，且大顶堆queMin最多比小顶堆queMax多一个元素。  

> 使用addNum操作加入一个新元素时，共有以下几种情况：  
1. 这个元素小于等于大顶堆queMin的队头，因此该元素小于中位数，会拉低中位数，将其放入大顶堆queMin。此外，查看两个堆的长度，若大顶堆queMin比小顶堆queMax多一个以上元素，则将大顶堆queMin的队头放入小顶堆queMax。  
2. 这个元素大于大顶堆queMin的队头，因此该元素大于中位数，会拉高中位数，将其放入小顶堆queMax。此外，查看两个堆的长度，若大顶堆queMin比小顶堆queMax更短，则将小顶堆queMax的队头放入大顶堆queMin。

> 使用findMedian操作时，根据两个堆的长度计算中位数。

> 注意，由于heapq只能维护小顶堆，因此在维护大顶堆queMin时需要将新加入的num取负号，维护一个小顶堆，使用queMin时在取负号变回原来的num。

## 复杂度

- 时间复杂度:addNum操作的复杂度为 $O(logn)$ ，其中 $n$ 是累积添加的数的数量；findMedian操作的复杂度为 $O(1)$ 。

- 空间复杂度: $O(n)$ ，其中 $n$ 是累积添加的数的数量。

> 优先队列的开销。

## 代码

In [1]:
import heapq

In [2]:
class MedianFinder:

    def __init__(self):
        # 元素个数为奇数时，queMin比queMax多一个数，queMin队头即为中位数
        # 元素个数为偶数时，queMin与queMax长度相同，两者队头元素的平均值即为中位数
        self.queMin = list()  # 大顶堆，值从左到右依次减小，其中的值小于等于中位数
        self.queMax = list()  # 小顶堆，值从左到右依次增加，其中的值大于等于中位数

    def addNum(self, num):
        queMin_ = self.queMin
        queMax_ = self.queMax

        # 判断queMin_是否为空，或num是否小于等于queMin_的队头元素
        if not queMin_ or num <= -queMin_[0]:
            # heappush为小顶堆操作，因此要取负号
            # 若num为最小值，直接heappush会将其放入队头，取负号后变为最大值，就会将其放入队尾
            # 负号能够将小顶堆操作变为大顶堆操作。这样维护的queMin_中全是负数，是一个小顶堆
            # 若将queMin_中所有数都取负号，则变为一个正确的大顶堆
            heapq.heappush(queMin_, -num)
            # 此时num小于等于中位数，需要将该数添加到queMin_中
            # num会拉低中位数的值
            # queMin_不能比queMax_多出1个以上的元素，因此要将多出的元素（queMin_中的队头）放到queMax_中
            if len(queMax_) + 1 < len(queMin_):
                heapq.heappush(queMax_, -heapq.heappop(queMin_))
        # 若num大于queMin_的队头元素
        else:
            # 将num压入queMax_
            heapq.heappush(queMax_, num)
            # num会拉高中位数的值
            # 由于queMax_的长度必须小于等于queMin_，因此要将多出的元素（queMax_中的队头）放到queMin_中
            if len(queMax_) > len(queMin_):
                heapq.heappush(queMin_, -heapq.heappop(queMax_))
        
    def findMedian(self):
        queMin_ = self.queMin
        queMax_ = self.queMax

        # 如果queMin_的长度大于queMax_，则queMin_的队头即为中位数
        if len(queMin_) > len(queMax_):
            return -queMin_[0]
        # 否则需要计算两者队头的平均
        return (-queMin_[0] + queMax_[0]) / 2

#### 测试一

In [3]:
newMedianFinder = MedianFinder()
newMedianFinder.addNum(1)
print(newMedianFinder.queMax, newMedianFinder.queMin)
newMedianFinder.addNum(2)
print(newMedianFinder.queMax, newMedianFinder.queMin)
print(newMedianFinder.findMedian())
newMedianFinder.addNum(3)
print(newMedianFinder.queMax, newMedianFinder.queMin)
print(newMedianFinder.findMedian())

[] [-1]
[2] [-1]
1.5
[3] [-2, -1]
2
